In [11]:
import numpy as np
import pandas as pd
import scipy
import scipy.stats as ss
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib_inline import backend_inline
from statsmodels.sandbox.regression.predstd import wls_prediction_std
backend_inline.set_matplotlib_formats('svg')


In [62]:
#1
data = pd.read_csv('Advertising.csv')

def f(a, b, c, d):
    def reg(x):
        tv, radio = x
        return - (a + b * tv + c * radio + d * tv * radio)
    return reg

def minimize_budget(budget, reg):
    sum_ = scipy.optimize.LinearConstraint(np.ones(2), budget, budget)
    eps = 0.00000001
    bounds = [(eps, budget - eps) for i in range(2)]
    x0 = np.array([budget/2] * 2)
    res = scipy.optimize.minimize(reg, x0=x0, bounds=bounds, constraints=sum_)
    return res

TV = data['TV']
radio = data['radio']
budget = TV + radio + data['newspaper']

y = data['sales']
X = sm.add_constant(pd.concat([data[['TV', 'radio']], TV * radio], axis=1))

model = sm.OLS(y, X).fit()
print(model.summary(slim=True))
print(f"params {model.params}")

a, b, c, d = model.params
new_tv, new_radio, prod = [], [], []

for bud in budget:
    res = minimize_budget(bud, f(a, b, c, d))
    new_tv.append(res.x[0])
    new_radio.append(res.x[1])
    prod.append(res.x[0] * res.x[1])
    
prod = pd.DataFrame({'prod': prod})
new_tv = pd.DataFrame({'TV': new_tv})
new_radio = pd.DataFrame({'radio': new_radio})

new_x = sm.add_constant(pd.concat([new_tv, new_radio, prod], axis=1))
new_sales = model.predict(new_x)


prstd, iv_l, iv_u = wls_prediction_std(model, new_x)

variances = prstd ** 2

var = np.sqrt(sum(variances))
var
n = len(y)
k = 3
z = ss.t(n - k - 1).ppf(0.975)

sum(new_sales) - z * var, sum(new_sales) + z*var

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.968
Model:                            OLS   Adj. R-squared:                  0.967
No. Observations:                 200   F-statistic:                     1963.
Covariance Type:            nonrobust   Prob (F-statistic):          6.68e-146
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.7502      0.248     27.233      0.000       6.261       7.239
TV             0.0191      0.002     12.699      0.000       0.016       0.022
radio          0.0289      0.009      3.241      0.001       0.011       0.046
0              0.0011   5.24e-05     20.727      0.000       0.001       0.001

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is lar

(4947.505794493587, 5006.228247649993)

In [106]:
#2
data = pd.read_csv('Advertising.csv')

def generate_e(e):
    return np.random.choice(e, len(e))

def get_beta(X, model_, real_e):
    e = generate_e(real_e)
    y = model_.predict(X) + e
    model = sm.OLS(y, X).fit()
    return model.params

TV = data['TV']
radio = data['radio']
budget = TV + radio + data['newspaper']

y = data['sales']
X = sm.add_constant(pd.concat([data[['TV', 'radio']], TV * radio], axis=1))

model = sm.OLS(y, X).fit()
#print(model.summary(slim=True))
#print(f"params {model.params}")

e = y - model.predict(X)
#e, model.resid
#model.predict(X), model.fittedvalues
beta = model.params

radios = []
n = 10000
for i in range(n):
    radios.append(get_beta(X, model, e)['radio'])

np.quantile(radios, q=0.025), np.quantile(radios, q=0.975)


(0.011495456994453116, 0.04607764849285114)

**Задача 3**

$$y = \gamma + \epsilon, E \epsilon = 0$$

$$P_0, P_1 - \text{проекции на $L_0$ и $L_1$ соответственно (линейные операторы)}$$

$$P_0 \gamma = \gamma = P_1 \gamma, \text{ т.к. } \gamma \in L_1 \subset L_0$$

$$ D_0 = y - P_0 y = \gamma + \epsilon - P_0(\gamma + \epsilon) = \gamma + \epsilon - P_0\gamma - P_0\epsilon = \epsilon - P_0\epsilon = (I - P_0) \epsilon \bot L_0$$

$$D_{01} = P_0 y - P_1 y = P_0\gamma + P_0\epsilon - P_1\gamma - P_1\epsilon =  P_0\epsilon - P_1\epsilon = (P_0 - P_1) \epsilon \in L_0$$

$$\text{то есть } D_0 \bot D_{01}$$

$$ED_0 = E[(I - P_0) \epsilon] = (I - P_0)E \epsilon = 0$$
$$ED_1 = E[(P_0 - P_1) \epsilon] = (P_0 - P_1)E \epsilon = 0$$


**задача 4**

*1*

$$R^n = L_0 \oplus L_{\bot}$$

$$RSS = || y - x \beta^* ||^2 = || y - hy ||^2 = || (I - h)y ||^2 = ||D_0||^2$$

$$D_0 \bot L_0 \text{ по 3 } \Rightarrow (I - h)y \bot L_0  \Rightarrow (I - h)y \in L_{\bot}$$

$$\prod_i := (I - h) $$

$$ED_0 = 0 \Rightarrow E(\prod_i y) = 0$$

$$RSS = || \prod_iy||^2 = || \prod_i y - E(\prod_iy) || ^ 2$$

$$dim(L_0) = k + 1 \Rightarrow dim(L_{\bot}) = n - k - 1$$

$$\text{по теореме с пререквизита } \frac{RSS}{\sigma^2} \sim \chi^2(n - k - 1)$$

$$RSS \sim \sigma^2 \chi^2(n - k - 1)$$

*2*




*3*

$$\beta^* - \beta = (x^Tx)^{-1}x^T\epsilon \sim N(0, cov(\beta^*))$$

$$cov(\beta^*) = \sigma^2(x^Tx)^{-1}$$

$$c(\beta^* - \beta) \sim N(0, \sigma^2 c (x^Tx)^{-1} c^T)$$

$$\hat{\sigma} = \sqrt{\frac{RSS}{n-k-1}} = \sigma \sqrt{\frac{\frac{RSS}{\sigma^2}}{n-k-1}}$$

$$\text{Тогда по предыдущим пунктам}$$

$$ Y := \frac{RSS}{\sigma^2} \sim \chi^2(n - k - 1)$$

$$X := \frac{c(\beta^* - \beta)}{ \sigma \sqrt{c (x^Tx)^{-1} c^T} } \sim N(0, 1)$$

$$X, Y \text{ - независимы }$$

$$\text{ тогда по пререквизитам } \frac{X}{ \sqrt{ \frac{Y}{n-k-1} } } \sim T(n-k-1)$$
